In [33]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [88]:
class wiki_scrape:

    def __init__(self):
        self.all_links = []

    def scrape_wiki(self, url, steps):
        if len(self.all_links)>10000:
            return None
        elif steps>8:
            return None
        data = requests.get(url).content
        soup = BeautifulSoup(data, 'html.parser')
        bodyContent = soup.find("div", {"id": "bodyContent"})
        temp_link_list = []
        for hyperlink in bodyContent.find_all('a'):
            link = hyperlink.get('href')
            if isinstance(link, str) and link[:5]=='/wiki':
                temp_link = f'https://en.wikipedia.org{link}'
                self.all_links.append([url, temp_link])
                temp_link_list.append(temp_link)
                if temp_link not in [i[0] for i in self.all_links]:
                    temp_link_list.append(temp_link)
        steps += 1
        for url in temp_link_list:
            self.scrape_wiki(url, steps)

In [85]:
obj = wiki_scrape()
obj.scrape_wiki('https://en.wikipedia.org/wiki/Earl_of_Dysart', 1)
df = pd.DataFrame(obj.all_links)
df.columns = ['source', 'destination']
df = df.drop_duplicates()
df.to_csv('wiki_links.csv', index=False)

In [99]:
for i in range(10000):
    checkpoint_df = pd.read_csv('wiki_links.csv')
    url = checkpoint_df.tail(1)['source'].values[0]
    obj = wiki_scrape()
    obj.scrape_wiki(url, 1)
    df = pd.DataFrame(obj.all_links)
    df.columns = ['source', 'destination']
    df = pd.concat([checkpoint_df, df], sort=False)
    df = df.drop_duplicates()
    df.to_csv('wiki_links.csv', index=False)
    print(f'checkpoint {i} shape {df.shape}')
    if len(checkpoint_df)==len(df):
        checkpoint_df = pd.read_csv('wiki_links.csv')
        url = checkpoint_df.tail(1000).head(1)['destination'].values[0]
        obj = wiki_scrape()
        obj.scrape_wiki(url, 1)
        df = pd.DataFrame(obj.all_links)
        df.columns = ['source', 'destination']
        df = pd.concat([checkpoint_df, df], sort=False)
        df = df.drop_duplicates()
        df.to_csv('wiki_links.csv', index=False)
        print(f'checkpoint {i} shape {df.shape}')
        if len(checkpoint_df)==len(df):
            checkpoint_df = pd.read_csv('wiki_links.csv')
            url = checkpoint_df.tail(100).head(1)['destination'].values[0]
            obj = wiki_scrape()
            obj.scrape_wiki(url, 1)
            df = pd.DataFrame(obj.all_links)
            df.columns = ['source', 'destination']
            df = pd.concat([checkpoint_df, df], sort=False)
            df = df.drop_duplicates()
            df.to_csv('wiki_links.csv', index=False)
            print(f'checkpoint {i} shape {df.shape}')
            if len(checkpoint_df)==len(df):
                break

checkpoint 0 shape (74063, 2)
checkpoint 0 shape (77578, 2)
checkpoint 1 shape (81906, 2)
checkpoint 2 shape (81906, 2)
checkpoint 2 shape (85503, 2)
checkpoint 3 shape (85503, 2)
checkpoint 3 shape (88691, 2)
checkpoint 4 shape (91751, 2)
checkpoint 5 shape (95347, 2)
checkpoint 6 shape (99167, 2)
checkpoint 7 shape (102070, 2)
checkpoint 8 shape (104591, 2)
checkpoint 9 shape (107993, 2)
checkpoint 10 shape (110503, 2)
checkpoint 11 shape (111659, 2)
checkpoint 12 shape (114037, 2)
checkpoint 13 shape (117652, 2)


KeyboardInterrupt: 